# Removing hard shadows from images

In [4]:
import numpy as np
import matplotlib.pyplot as plt
import cv2
import math

%matplotlib inline

In [6]:
# Read an Image from dataset
original_img = cv2.imread('MangoLeavesDatabase/Alphonso/front/alphonso_front_4unv4g3l1lwivsgov3xx1h8n.jpg')
# Convert color channels from BGR to RGB
img2rgb = cv2.cvtColor(original_img, cv2.COLOR_BGR2RGB)
# Apply K-Means to reduce color space in image
img_pixels = np.float32(img2rgb.reshape((-1, 3)))
criteria = (cv2.TERM_CRITERIA_EPS + cv2.TERM_CRITERIA_MAX_ITER, 5, 1.0)
K = 3 # no of clusters
ret,labels,centers = cv2.kmeans(img_pixels, K, None, criteria, 5, cv2.KMEANS_RANDOM_CENTERS)
centers = np.uint8(centers)
cluster_img = centers[labels.flatten()]
# Change all shadow bluish pixels to white in clustered image
for i,p in enumerate(cluster_img):
    max_intensity = max(p)
    if max_intensity == p[2]:
        cluster_img[i] = [255, 255, 255]
cluster_img = cluster_img.reshape(img2rgb.shape)
# Apply thresholding to clustered image
cluster_img2gray = cv2.cvtColor(cluster_img, cv2.COLOR_RGB2GRAY)
retval, th_img = cv2.threshold(cluster_img2gray, 0, 255, cv2.THRESH_BINARY_INV+cv2.THRESH_OTSU)
# Applying Closing Morphological operation on leaf
kernel = np.ones((5, 5),np.uint8)
th_img = cv2.morphologyEx(th_img, cv2.MORPH_CLOSE, kernel, iterations=3)
# Find contours in threshold image
contours, _ = cv2.findContours(th_img, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
if len(contours) != 0:
    # Get contour with largest area as leaf contour
    leaf_contour = max(contours, key=cv2.contourArea)
    # Get the bounding box for leaf contour
    x, y, w, h = cv2.boundingRect(leaf_contour)
    # Draw contours on original, clustered and threashold images
    img1 = cv2.drawContours(img2rgb, [leaf_contour], 0, (255,0,0), 2)
    img2 = cv2.drawContours(cluster_img, [leaf_contour], 0, (255,0,0), 2)
    img3 = cv2.drawContours(cv2.cvtColor(th_img, cv2.COLOR_GRAY2RGB), [leaf_contour], 0, (255, 0, 0), 2)
    # Display Results
    """titles = ['Original Image','K-Means Image', 'Threshold Image']
    imgs = [img1, img2, img3]
    plt.figure('Leaf', figsize=(18, 8))
    for i in range(0, 3):
        plt.subplot(3, 1, i+1)
        plt.axis('off')
        plt.title(titles[i])
        plt.imshow(imgs[i])
    plt.show()"""
    cv2.imshow('Original', img1)
    cv2.imshow('Clustered', img2)
    cv2.imshow('Thresh', img3)
    cv2.waitKey(0)
    cv2.destroyAllWindows()
else:
    print('No contour found in image')